In [1]:
import pandas as pd
import numpy as np

In [2]:
data1 = pd.read_csv("5 min traffic intensity 2016 2019 locaties aangepast 2.csv")
data2 = pd.read_csv("5 min traffic intensity 2016 2019 locatie test 3.csv")

In [3]:
stacked_data = pd.concat([data1, data2], axis=0)

# Reset index if needed
#stacked_data.reset_index(drop=True, inplace=True)

# Display the stacked DataFrame
stacked_data.head()

,id_meetlocatie,start_meetperiode,eind_meetperiode,ndw_index,gebruikte_minuten_intensiteit,gebruikte_minuten_snelheid,gem_intensiteit,gem_snelheid,totaal_aantal_rijstroken,rijrichting,technical_exclusion,traffic_flow_deviation_exclusions
0,RWS01_MONIBAS_0070vwb1401ra,2016-01-01 00:00:00,2016-01-01 00:05:00,F1B,0.0,0.0,-1.0,-1.0,1,eastBound,NaN,NaN
1,RWS01_MONIBAS_0070vwb1401ra,2016-01-01 00:00:00,2016-01-01 00:05:00,F2B,0.0,0.0,-1.0,-1.0,1,eastBound,NaN,NaN
2,RWS01_MONIBAS_0070vwb1401ra,2016-01-01 00:00:00,2016-01-01 00:05:00,F3B,0.0,0.0,-1.0,-1.0,1,eastBound,NaN,NaN
3,RWS01_MONIBAS_0070vwb1401ra,2016-01-01 00:00:00,2016-01-01 00:05:00,F4B,0.0,0.0,-1.0,-1.0,1,eastBound,NaN,NaN
4,RWS01_MONIBAS_0070vwb1401ra,2016-01-01 00:05:00,2016-01-01 00:10:00,F1B,0.0,0.0,-1.0,-1.0,1,eastBound,NaN,NaN


In [4]:
# Define the list of values to be removed
values_to_remove = ['RWS01_MONIBAS_0071hrl1805ra', 'RWS01_MONIBAS_0281hrl1476ra',
                    'RWS01_MONIBAS_0071hrr1805ra', 'RWS01_MONIBAS_0281hrr1476ra',
                    'RWS01_MONIBAS_0321hrr0654ra', 'RWS01_MONIBAS_0321hrl0654ra',
                    'RWS01_MONIBAS_0321hrr0467ra', 'RWS01_MONIBAS_0321hrl0467ra',
                    'RWS01_MONIBAS_0041hrl0151ra', 'RWS01_MONIBAS_0041hrr0152ra',
                    'RWS01_MONIBAS_0070vwb1401ra', 'RWS01_MONIBAS_0070vwc1401ra'
                   ]

# Filter out rows with specified values in the id_meetlocatie column
stacked_data = stacked_data[~stacked_data['id_meetlocatie'].isin(values_to_remove)]

# Reset index if needed
stacked_data.reset_index(drop=True, inplace=True)

# Display the filtered DataFrame
#filtered_stacked_data.head()

In [5]:
data = stacked_data

In [6]:
# Assuming 'data' is your DataFrame
data['gem_intensiteit'] = data['gem_intensiteit'].replace(-1, 0)
data['gem_snelheid'] = data['gem_snelheid'].replace(-1, 0)

In [7]:
#check for missing values

data.groupby('id_meetlocatie').apply(lambda x: x['gem_snelheid'].isnull().sum())

id_meetlocatie
RWS01_MONIBAS_0051hrl0060ra    0
RWS01_MONIBAS_0051hrr0060ra    0
RWS01_MONIBAS_0071hrl1466ra    0
RWS01_MONIBAS_0071hrl1933ra    0
RWS01_MONIBAS_0071hrr1466ra    0
RWS01_MONIBAS_0071hrr1933ra    0
RWS01_MONIBAS_0281hrl1857ra    0
RWS01_MONIBAS_0281hrr1857ra    0
dtype: int64

In [8]:
data["ndw_index"].unique()

array(['F1B', 'F2B', 'F3B', 'F4B', 'F9B', 'F10B', 'F11B', 'F12B', 'F1A',
       'F3A'], dtype=object)

In [9]:
#remove stopping lane

data = data[data['ndw_index'] != 'F801A']

In [10]:
# Assuming 'data' is your DataFrame and 'time_column' is the name of the column containing the datetime values
data['time_column'] = pd.to_datetime(data['start_meetperiode'])  # Convert to datetime if not already

# Extract year, month, day, hour, and minute into separate columns
data['date'] = data['time_column'].dt.date
data['year'] = data['time_column'].dt.year
data['month'] = data['time_column'].dt.month
data['day'] = data['time_column'].dt.day
data['hour'] = data['time_column'].dt.hour
data['minute'] = data['time_column'].dt.minute
data['day_of_week'] = data['time_column'].dt.dayofweek
data['weekend'] = (data['day_of_week'] >= 5).astype(int)

# Now you have separate columns for year, month, day, hour, and minute

In [11]:
# Add 0.000001 to the values in the "gem_intensiteit" column
data['gem_intensiteit'] += 0.000001

In [12]:
data['intensiteit'] = data['gem_intensiteit']/12

In [13]:
data['sumspeed'] = data['intensiteit'] * data ['gem_snelheid']

In [14]:
#extra check not necessary if previous one was already succesful
#data.groupby('id_meetlocatie').apply(lambda x: x['gem_snelheid'].isnull().sum())

In [15]:
# Select the columns to group by
group_columns = ['id_meetlocatie', 'date', 'year', 'month', 'day', 'hour', 'minute', 'day_of_week', 'weekend', 'totaal_aantal_rijstroken']

# Group by the selected columns and aggregate other columns
grouped_data = data.groupby(group_columns).agg({'intensiteit': 'sum', 'sumspeed': 'sum'}).reset_index()

In [16]:
grouped_data['avg_speed'] = grouped_data['sumspeed'] / grouped_data['intensiteit']
grouped_data = grouped_data.drop(columns=['sumspeed'])

In [17]:
grouped_data.head(20)

,id_meetlocatie,date,year,month,day,hour,minute,day_of_week,weekend,totaal_aantal_rijstroken,intensiteit,avg_speed
0,RWS01_MONIBAS_0051hrl0060ra,2016-01-01,2016,1,1,0,0,4,0,2,4.0,107.439998
1,RWS01_MONIBAS_0051hrl0060ra,2016-01-01,2016,1,1,0,5,4,0,2,11.0,106.048182
2,RWS01_MONIBAS_0051hrl0060ra,2016-01-01,2016,1,1,0,10,4,0,2,5.0,116.999998
3,RWS01_MONIBAS_0051hrl0060ra,2016-01-01,2016,1,1,0,15,4,0,2,6.0,109.599998
4,RWS01_MONIBAS_0051hrl0060ra,2016-01-01,2016,1,1,0,20,4,0,2,5.0,112.929998
5,RWS01_MONIBAS_0051hrl0060ra,2016-01-01,2016,1,1,0,25,4,0,2,7.0,114.349999
6,RWS01_MONIBAS_0051hrl0060ra,2016-01-01,2016,1,1,0,30,4,0,2,9.0,110.866667
7,RWS01_MONIBAS_0051hrl0060ra,2016-01-01,2016,1,1,0,35,4,0,2,12.0,106.820000
8,RWS01_MONIBAS_0051hrl0060ra,2016-01-01,2016,1,1,0,40,4,0,2,15.0,111.320000
9,RWS01_MONIBAS_0051hrl0060ra,2016-01-01,2016,1,1,0,45,4,0,2,27.0,115.515926


In [18]:
grouped_data.shape

(3212544, 12)

In [19]:
'''test to see whether null variables'''
#grouped_data.groupby('id_meetlocatie').apply(lambda x: x['avg_speed'].isnull().sum())

'test to see whether null variables'

In [20]:
#grouped_data['hour'] = pd.to_numeric(grouped_data['hour'], errors='coerce')

In [21]:
#grouped_data = grouped_data[(grouped_data['hour'] > 5)] 

In [22]:
#grouped_data = grouped_data[(grouped_data['hour'] < 21)]

In [23]:
holidays = pd.read_csv("holidays.csv")

In [24]:
import pandas as pd

# Assuming holidays is your DataFrame and 'date' is the column containing dates
# Convert 'date' column to datetime object
holidays['date'] = pd.to_datetime(holidays['date'], errors='coerce')

# Check unique values and infer date format
unique_dates = holidays['date'].dropna().dt.strftime('%Y-%m-%d').unique()
date_format = unique_dates[0] if len(unique_dates) == 1 else "Multiple formats detected"

# Print the inferred date format
print("Inferred date format:", date_format)

Inferred date format: Multiple formats detected


In [25]:
grouped_data['date'] = pd.to_datetime(grouped_data['date'], errors='coerce')

# Check unique values and infer date format
unique_dates = grouped_data['date'].dropna().dt.strftime('%Y-%m-%d').unique()
date_format = unique_dates[0] if len(unique_dates) == 1 else "Multiple formats detected"

# Print the inferred date format
print("Inferred date format:", date_format)

Inferred date format: Multiple formats detected


In [26]:
# Round the 'intensiteit' and 'alg_speed' columns to two decimals
grouped_data['intensiteit'] = grouped_data['intensiteit'].round(2)
grouped_data['avg_speed'] = grouped_data['avg_speed'].round(2)


In [27]:
merged_data = pd.merge(grouped_data, holidays, on='date', how='left')

merged_data['is_public_holiday'] = merged_data['is_public_holiday'].fillna(0).astype(int)

In [28]:
merged_data.head()

,id_meetlocatie,date,year,month,day,hour,minute,day_of_week,weekend,totaal_aantal_rijstroken,intensiteit,avg_speed,is_public_holiday
0,RWS01_MONIBAS_0051hrl0060ra,2016-01-01,2016,1,1,0,0,4,0,2,4.0,107.44,1
1,RWS01_MONIBAS_0051hrl0060ra,2016-01-01,2016,1,1,0,5,4,0,2,11.0,106.05,1
2,RWS01_MONIBAS_0051hrl0060ra,2016-01-01,2016,1,1,0,10,4,0,2,5.0,117.00,1
3,RWS01_MONIBAS_0051hrl0060ra,2016-01-01,2016,1,1,0,15,4,0,2,6.0,109.60,1
4,RWS01_MONIBAS_0051hrl0060ra,2016-01-01,2016,1,1,0,20,4,0,2,5.0,112.93,1


In [29]:
merged_data.to_csv('Preprocessed_5min_2016_2019_altLocations.csv', index=False)